# Anonimización de Datos en Data Science

Método que, a diferencia de la pseudonimización, **elimina permanentemente cualquier información que pudiera identificar a una persona**, preservando sólo aquellos datos que son necesarios para la investigación. Es la opción por defecto para estudios que buscan resultados generalizables. Datos como nombre, apellido, código de documento ID, o número de teléfono, etc. **no son relevantes** para asuntos estadísticos reales.

In [1]:
from typing import Any

import numpy as np
import pandas as pd

## Caso de ejemplo

**NOTA PERSONAL:** Los datos aquí trabajados (proporcionados por Federico) son obviamente de tipo _dummy_ (no reales). Sin embargo, para complementar la información dada en este curso, debe considerarse que, en el mundo real, los datos pueden ser proveídos directamente desde fuentes tales como archivos locales (CSV, XLSX, etc.) o una **base de datos** local; o bien, indirectamente desde servicios online a través de una **API**. Sobre esto último, es conveniente tomar algunas precauciones tales como asegurarse de usar siempre **conexiones seguras**, y de tener un manejo estrictamente cuidadoso de los datos sensibles como las credenciales de autenticación, las claves de las API, y otras **variables de entorno**, a fin de evitar cualquier fuga de datos. 🚨

In [2]:
names: list[str] = ['Ana', 'Juan', 'Luis', 'Pedro', 'Silvina']
emails: list[str] = [
    'ana@ejemplo.com',
    'juan@ejemplo.com',
    'luis@ejemplo.com',
    'pedro@ejemplo.com',
    'silvina@ejemplo.com'
]
ages: list[int] = [22, 37, 15, 49, 63]
cities: list[str] = [
    'Ciudad A',
    'Ciudad B',
    'Ciudad C',
    'Ciudad D',
    'Ciudad E'
]
salaries: list[int] = [55000, 34000, 76000, 51000, 62000]
banks: list[str] = ['Banco 1', 'Banco 3', 'Banco 1', 'Banco 2', 'Banco 3']

data: dict[str, Any] = {
    'Nombre': names,
    'Email': emails,
    'Edad': ages,
    'Ubicación': cities,
    'Salario': salaries,
    'Banco': banks
}

df: pd.DataFrame = pd.DataFrame(data)
df

,Nombre,Email,Edad,Ubicación,Salario,Banco
0,Ana,ana@ejemplo.com,22,Ciudad A,55000,Banco 1
1,Juan,juan@ejemplo.com,37,Ciudad B,34000,Banco 3
2,Luis,luis@ejemplo.com,15,Ciudad C,76000,Banco 1
3,Pedro,pedro@ejemplo.com,49,Ciudad D,51000,Banco 2
4,Silvina,silvina@ejemplo.com,63,Ciudad E,62000,Banco 3


Primero, eliminemos nombres y correos electrónicos:

In [3]:
df.drop(['Nombre', 'Email'], axis=1, inplace=True)
df

,Edad,Ubicación,Salario,Banco
0,22,Ciudad A,55000,Banco 1
1,37,Ciudad B,34000,Banco 3
2,15,Ciudad C,76000,Banco 1
3,49,Ciudad D,51000,Banco 2
4,63,Ciudad E,62000,Banco 3


Con esto, ya se ha logrado anonimizar las instancias en gran medida, pero no es la única medida y se puede proceder aún más para garantizar la anonimización al máximo.

**NOTA DE FEDERICO:** Una estrategia de anonimización es el **redondeo** o **truncado** de datos numéricos tales como edades, fechas, números de documentos, etc., ya que podemos seguir investigando aspectos generales de la información, pero sin tener la información demasiado específica.

In [4]:
# Transformar edades a sus respectivas décadas
df['Edad'] = (df['Edad'] // 10) * 10
df

,Edad,Ubicación,Salario,Banco
0,20,Ciudad A,55000,Banco 1
1,30,Ciudad B,34000,Banco 3
2,10,Ciudad C,76000,Banco 1
3,40,Ciudad D,51000,Banco 2
4,60,Ciudad E,62000,Banco 3


En el ejemplo, ya no es que la persona en el índice `0` tenga `22` años (dato específico), sino que "está en sus veintes" (dato inespecífico, pero útil para efectos de investigación y que, dicho sea de paso, facilita enormemente el análisis por rangos etáreos).

**NOTA DE FEDERICO:** Otra estrategia de anonimización de datos numéricos es añadir **ruido aleatorio** para desvincularlos de su valor real, útil para ofuscar valores exactos o patrones específicos en el conjunto de datos, aunque preservando la utilidad estadística. Es decir, **modifica los datos individuales, pero mantiene las propiedades generales** del conjunto (tales como la media, la desviación estándar, la varianza, etc.).


**NOTA PERSONAL:** En el habla cotidiana, "ruido" es prácticamente sinónimo de cualquier cosa "aleatoria" (como en el caso del [ruido blanco](https://es.wikipedia.org/wiki/Ruido_blanco)), pero debemos recordar que, en la jerga técnica, sí hay una distinción muy importante entre lo "aleatorio" y lo "pseudoaleatorio", en donde el último implica cierto determinismo (y, por ende, probabilidades de trazabilidad).

In [5]:
# Generar ruido, manteniendo distribución normal
noise: np.ndarray = np.random.normal(0, 10000, size=df['Salario'].shape)
noise

array([ -1398.150668  , -18640.42613047,  10925.14896557,  12413.93708749,
       -15703.3832971 ])

In [6]:
# Ofuscar con ruido
df['Salario'] += noise
df

,Edad,Ubicación,Salario,Banco
0,20,Ciudad A,53601.849332,Banco 1
1,30,Ciudad B,15359.573870,Banco 3
2,10,Ciudad C,86925.148966,Banco 1
3,40,Ciudad D,63413.937087,Banco 2
4,60,Ciudad E,46296.616703,Banco 3


**NOTA DE FEDERICO:** Otra estrategia de anonimización para datos de tipo cadena es la de **permutación** o **shuffling** y consiste en cambiar de orden ciertos datos, de modo que se pierda la relación que hay entre esos valores con otros campos del dataset.

Supongamos que no queremos que se sepa en qué banco cada uno de sus usuarios cobra su sueldo:

In [7]:
df['Banco'] = np.random.permutation(df['Banco'])
df

,Edad,Ubicación,Salario,Banco
0,20,Ciudad A,53601.849332,Banco 1
1,30,Ciudad B,15359.573870,Banco 3
2,10,Ciudad C,86925.148966,Banco 3
3,40,Ciudad D,63413.937087,Banco 1
4,60,Ciudad E,46296.616703,Banco 2


**NOTA DE FEDERICO:** ¡La permutación sólo puede implementarse cuando la relación entre la columna objetivo y otras NO sea relevante para los objetivos del estudio! ☝